# SimpleRNNLMクラスを実装する
SimpleRNNLMクラスは、RNNレイヤを用いた言語モデルを構築するためのクラスである

In [ ]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.py, layers.py, time_layers.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')


from common.time_layers import TimeEmbedding, TimeRNN, TimeAffine, TimeSoftmaxWithLoss


Google Colab. 上での実行です
「ファイルを選択」から、notebook/commonフォルダのfunctions.py, layers.py, time_layers.pyを選択し、アップロードしてください


Saving functions.py to functions.py
Saving layers.py to layers.py
Saving time_layers.py to time_layers.py


### [演習]
* 以下のSimpleRNNLMのクラスを完成させましょう

In [ ]:
class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # 乱数生成関数を別名で定義する
        rn = np.random.randn # 標準正規分布からサンプリングする関数

        # パラメータの初期化
        embed_W = rn(V, D) / 100 # 小さな値で初期化する
        rnn_Wx = rn(D, H) * np.sqrt(2/(D+H)) # Xavierの初期値
        rnn_Wh = rn(H, H) * np.sqrt(2/(H+H)) # Xavierの初期値
        rnn_b = np.zeros(H)
        affine_W = rn(H, V) * np.sqrt(2/(H+V)) # Xavierの初期値
        affine_b = np.zeros(V)

        # レイヤの生成
        self.layers = [
            TimeEmbedding(embed_W), # 単語埋め込みレイヤ
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True), # RNNレイヤ. 中間層の状態を引き継ぐため、statefulをTrueにしておく
            TimeAffine(affine_W, affine_b) # Affineレイヤ
        ]
        self.loss_layer = TimeSoftmaxWithLoss() # 損失レイヤ

        # すべてのパラメータと勾配をそれぞれ1つのリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params # リストを結合する
            self.grads += layer.grads # リストを結合する

    def predict(self, xs):
        """
        予測関数
        """
        # 損失レイヤ以外の全てのレイヤについて計算する
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs
    
    def forward(self, xs, ts):
        """
        順伝播計算
        xs : 入力の単語ID, 配列形状は(ミニバッチ数、時間数)
        """
        # 損失レイヤ以外の全てのレイヤについて計算する
        xs = self.predict(xs)
        
        # 損失レイヤを計算する
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        """
        逆伝播計算
        """
        # 損失レイヤの逆伝播計算
        dout = self.loss_layer.backward(dout)
        
        # 損失レイヤ以外の逆伝播計算
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
            
        return dout

In [ ]:
V = 10 # 語彙数
D = 3 # 埋め込みベクトルのノード数
H = 3 # RNNレイヤの隠れ層のノード数
srnnlm = SimpleRnnlm(V, D, H)

N = 2 # バッチサイズ
T = 4 # 時間数

# 単語ID
xs = np.array([[2, 5, 1, 4],[3, 2, 7, 8]]) # 入力の単語ID
ts = np.array([[5, 1, 4, 3],[2, 7, 8, 0]]) # 正解の単語ID

# 順伝播計算
loss = srnnlm.forward(xs, ts)
print("loss=", loss)
print()

# 逆伝播計算
dout = srnnlm.backward()
print("dout=", dout) # 最後の層はEmbeddingレイヤなので、Noneが返ってくる
print()

# 勾配の確認
print("Embeddingレイヤの重み行列=\n", srnnlm.layers[0].grads)

loss= 2.3027853638669784

dout= None

Embeddingレイヤの重み行列=
 [array([[ 0.        ,  0.        ,  0.        ],
       [-0.12951273, -0.06205662, -0.13656104],
       [ 0.03969958,  0.03656063,  0.1140927 ],
       [ 0.07924961,  0.02334802,  0.02376232],
       [ 0.02932222,  0.00302022, -0.01292343],
       [ 0.03222273,  0.02504289,  0.07585042],
       [ 0.        ,  0.        ,  0.        ],
       [-0.03187653, -0.01042363, -0.01505843],
       [ 0.04152564,  0.01641677,  0.02769354],
       [ 0.        ,  0.        ,  0.        ]])]
